In [18]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# ChromeDriver 的下載管理工具
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 取得系統時間的工具
from datetime import datetime

# 強制停止/強制等待 (程式執行期間休息一下)
from time import sleep

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

import os

# 隨機取得 User-Agent
from fake_useragent import UserAgent
ua = UserAgent()

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                # 不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         # 最大化視窗
my_options.add_argument("--incognito")               # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")   # 取消通知
my_options.add_argument(f'--user-agent={ua.random}') # (Optional)加入 User-Agent

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
)

# 儲存資料的變數
list_data = []

# 走訪網址
url = 'https://consumer.fda.gov.tw/Food/TFND.aspx?nodeID=178'
type_index = 18
keyword = '冷凍'

In [20]:
#visit url page
def visit():
    driver.get(url);

#select drop-down menu:
def setDropDownMenu(type_index):
    try:
        #強制等待
        sleep(1)

        # 選擇食物類別
        type = Select(driver.find_element(By.CSS_SELECTOR, 'li > select'))
        type.select_by_index(type_index)
                      
        # 強制等待
        sleep(2)
    except Exception as err:
        print(str(err))

def key_in_search(keyword):
    try:
        search_box = driver.find_element(
            By.CSS_SELECTOR, 'input#ctl00_content_SelectK'
        )
        search_box.send_keys(keyword)

        # 強制等待
        sleep(1)
        
        # 按下查詢
        driver.find_element(
            By.CSS_SELECTOR, 'input#ctl00_content_searchBut'
        ).click()

        # 強制等待
        sleep(2)
    except Exception as err:
            print(str(err))

visit()
setDropDownMenu(type_index)
key_in_search(keyword)

## 開啟分頁

In [21]:
# 連續開 3 個分頁 (javascript)
for i in range(3):
    driver.execute_script(f'window.open("");')

In [22]:
# 切換到初始分頁 (索引為 0)
driver.switch_to.window(driver.window_handles[0])

In [23]:
# 將所有 tabs 轉址，以便取得對應列表; 0 不變
for index in range(1, len(driver.window_handles)):
    #change tabs
    driver.switch_to.window(
        driver.window_handles[index]
    )
    # 使分頁自動連結到指定網址 (此時的 drive 變數指向切後的分頁)
    driver.get(f'https://consumer.fda.gov.tw/Food/TFND.aspx?nodeID=178&t={type_index}&k={keyword}&p={index}')  

    # 強制等待
    sleep(3)

In [24]:
# 取得分頁列表
for index in range(1, len(driver.window_handles)):
    #change tabs
    driver.switch_to.window(driver.window_handles[index])

    #取得列表連結與內文
    for a in driver.find_elements(By.CSS_SELECTOR, 'td[data-th="樣品名稱"]>a'):
        list_data.append({
            'Food': a.get_attribute('innerText'),
            'Link': a.get_attribute('href')
        })

In [25]:
# 讀最後一頁開始，把所有分頁關掉 (初始頁要保留)
while len(driver.window_handles) > 1:
    #change tabs
    driver.switch_to.window(
        driver.window_handles[ len(driver.window_handles) - 1 ]
    )

    #close tabs
    driver.close()

In [26]:
#close browser
driver.quit()

In [27]:
#view list
list_data

[{'Food': '冷凍火腿炒飯',
  'Link': 'https://consumer.fda.gov.tw/Food/tfndDetail.aspx?nodeID=178&f=0&id=1753'},
 {'Food': '冷凍蝦仁炒飯',
  'Link': 'https://consumer.fda.gov.tw/Food/tfndDetail.aspx?nodeID=178&f=0&id=1770'},
 {'Food': '冷凍筒仔米糕',
  'Link': 'https://consumer.fda.gov.tw/Food/tfndDetail.aspx?nodeID=178&f=0&id=1797'},
 {'Food': '冷凍芝麻湯圓',
  'Link': 'https://consumer.fda.gov.tw/Food/tfndDetail.aspx?nodeID=178&f=0&id=1755'},
 {'Food': '冷凍花生湯圓',
  'Link': 'https://consumer.fda.gov.tw/Food/tfndDetail.aspx?nodeID=178&f=0&id=1756'},
 {'Food': '冷凍豬肉湯圓',
  'Link': 'https://consumer.fda.gov.tw/Food/tfndDetail.aspx?nodeID=178&f=0&id=1774'},
 {'Food': '冷凍烏龍麵',
  'Link': 'https://consumer.fda.gov.tw/Food/tfndDetail.aspx?nodeID=178&f=0&id=1972'},
 {'Food': '冷凍牛肉水餃',
  'Link': 'https://consumer.fda.gov.tw/Food/tfndDetail.aspx?nodeID=178&f=0&id=1784'},
 {'Food': '冷凍豬肉水餃',
  'Link': 'https://consumer.fda.gov.tw/Food/tfndDetail.aspx?nodeID=178&f=0&id=1771'},
 {'Food': '冷凍豬肉韭菜水餃',
  'Link': 'https://consum